In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import uvicorn

app = FastAPI()

# Carregamento do modelo (idealmente feito no startup, mas ok aqui para scripts simples)
model = joblib.load('modelo_sentimento.joblib')

# 1. Definição do Modelo de Dados (Schema)
# Isso garante que o JSON recebido tenha o campo 'text' e que seja uma string.
class InputData(BaseModel):
    text: str

@app.post('/predict')
def predict(data: InputData):
    # Acessamos os dados diretamente como atributos do objeto validado
    texto = data.text

    # Lógica de predição (Scikit-learn/Joblib geralmente bloqueiam a CPU,
    # por isso usamos 'def' normal em vez de 'async def' para que o FastAPI
    # rode isso em uma thread separada)
    prediction = model.predict([texto])[0]
    proba = model.predict_proba([texto]).max()

    # Retorno direto de dicionário (FastAPI converte para JSON automaticamente)
    return {
        "previsao": prediction,
        "probabilidade": float(proba)
    }

if __name__ == '__main__':
    # FastAPI precisa do Uvicorn para rodar
    uvicorn.run(app, host="0.0.0.0", port=8000)